### Purpose of this notebook
The purpose of this notebook is to download WLASL dataset and upload to S3 bucket after processing as Numpy

The videos will be labeled by the 'gloss' name from the Json.
Some videos only have the required sign between Frame-A and Frame-B. These will be cropped accordingly

In [1]:
#Install missing packages
#!pip install boto3
#!pip install pyarrow
#!pip install fastparquet
#!pip install s3fs
#!pip install mediapipe
#!pip install kaggle

In [2]:
# Import required libraries
import boto3 #Video files get read through this
import cv2
import os
import pandas as pd
import math
import numpy as np
import random
import io
import s3fs # Parquet files get read through this
import zlib # For compression
import time # To calculate download time
import configparser
import requests
import psutil # Checks memory usage
import tempfile
import json
import yt_dlp
import subprocess
from tqdm import tqdm
#import mediapipe as mp


In [3]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [4]:
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
prefix = '/msasl/RGB/'
save_path = '/content/temp_folder'
s3_URI = 's3://asl-capstone/msasl/RGB/'

In [5]:
# Create an s3 object
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')

In [6]:
# Create an S3FS object
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key) # Define an S3FS object

In [7]:
# # Let's read the WLASL json file to map the number to the word
# filename = "wlasl-dataset/WLASL_v0.3.json"
# wlasl_df = pd.read_json(filename)

In [8]:
# display(wlasl_df.head())
# print(wlasl_df.shape)
# #print(wlasl_df['instances'][0])

In [9]:
# # Not all videos from the JSON exist. Let's create a function to check which ones exist
# def video_features(json_data):
#     video_list = []
#     frame_list = []
#     gloss = json_data['gloss']
#     json_list = json_data['instances']
#     for id in json_list:
#         vid = id['video_id']
#         frame_start = id['frame_start']
#         frame_end = id['frame_end']
#         bbox = id['bbox']
#         if os.path.exists(f"wlasl-dataset/videos/{vid}.mp4"):
#             features = {'word':gloss,'id':vid,'start':frame_start, 'end':frame_end,'box':bbox}
#             #yield features
#             video_list.append(features)
#     return video_list

In [10]:
# # Load the data and get the features
# with open("wlasl-dataset/WLASL_v0.3.json",'r') as file:
#     json_data = file.read()
# json_data = json.loads(json_data)

In [11]:
# # Let's extract all relevant features from the JSON
# video_list = []
# for data in json_data:
#     video_list.append(video_features(data))

In [12]:
# # Let's check how many unique words we have
# word_list = []
# for video in video_list:
#     for vid in video:
#         word_list.append(vid['word'])

In [13]:
# print(len(word_list))
# word_list = list(set(word_list))
# print(len(word_list))
# print(word_list[:10])

In [14]:
# # Let's now work on the MS-ASL Dataset
# filenames = ['MSASL/MSASL_train.json','MSASL/MSASL_val.json','MSASL/MSASL_test.json']
# msasl_words = []
# for file in filenames:
#     with open("MSASL/MSASL_train.json",'r') as file:
#         json_data = file.read()
#     json_data = json.loads(json_data)
#     for word in json_data:
#         msasl_words.append(word)

In [15]:
# print(msasl_words[0])

In [16]:
# def msasl_features(msasl_word):
#     gloss = msasl_word['clean_text']
#     url = msasl_word['url']
#     frame_start = msasl_word['start_time']*msasl_word['fps']
#     frame_end = msasl_word['end_time']*msasl_word['fps']
#     bbox = msasl_word['box']
#     features = {'word':gloss,'url':url,'start':frame_start, 'end':frame_end,'box':bbox}
#     return features
    

In [17]:
# msasl_data = [] # Store all relevant information about the MSASL dataset
# for word in msasl_words:
#     msasl_data.append(msasl_features(word))    

In [18]:
# print(msasl_data[:4])

In [19]:
# matching_data = [x for x in msasl_data if x['word'] in word_list]
# print(matching_data[:4])

In [20]:
# print(len(matching_data))

In [21]:
# # Sort all the elements by the word so that they come sequentially
# matching_data = sorted(matching_data, key = lambda x: x['word'])

In [22]:
# print(matching_data[:10])

In [23]:
with fs.open(f"s3://asl-capstone/msasl/matching_words.parquet","rb") as f: # Read the data from S3
    matching_data = pd.read_parquet(f)

In [24]:
matching_data = matching_data.to_dict('records') # Convert to a list of dictionaries

In [25]:
# We need only a max of 20 videos per word
collected_words = []
matching_word = ""
counter = 0
for data in matching_data:
    #print(data['word'])
    if data['word'] != matching_word:
        matching_word = data['word']
        counter = 0
    if counter < 20:
        collected_words.append(data)
        counter+=1

In [26]:
print(len(collected_words))

16473


In [27]:
# Let's visualise the data
words_df = pd.DataFrame(collected_words)
word_counts = words_df['word'].value_counts()

In [28]:
with fs.open(f"s3://asl-capstone/msasl/master_file_Deanna_1.parquet","rb") as f:
    read_words = pd.read_parquet(f)

In [29]:
print(len(read_words))
read_words['caption'].value_counts()

2414


caption
quiet         22
ring          21
spain         20
strawberry    20
store         20
              ..
table          8
psychology     6
spell          3
purchase       3
popcorn        3
Name: count, Length: 127, dtype: int64

In [30]:
collected_words = collected_words[440:] # Drop the first 440 words since those were already processed

In [31]:
len(collected_words)

16033

In [32]:
def check_video(url): #Checks if a YouTube video is available and returns True if available and False if not
    try:
        result = subprocess.run(["yt-dlp", "--simulate", "--get-url", url], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE);
        return True
    except subprocess.CalledProcessError:
        return False # Video is not available in YouTube

In [33]:
def download_vid(url, filename):
    try:
        filename = "temp_folder/"+filename
        if os.path.exists(filename):
            os.remove(filename)
            # print("Deleted file")
        if not os.path.exists(filename):
            command = ["yt-dlp","-o",filename,url]
            subprocess.run(command);
            return True
    except:
        return False
        

In [34]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    if x > y:
        start_x = (x-y)/2
        end_x = start_x + y
        start_x = int(start_x)
        end_x = int(end_x)
        return frame[:, int(start_x):int(end_x)]
    else:
        return frame

In [35]:
def load_video(filename, max_frames=0, resize=(256,256)):
    filename = "temp_folder/"+filename
    video_capture = cv2.VideoCapture(filename)
    frames = []
    try:
      while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
          break
        frame = crop_center_square(frame)
        frame = cv2.resize(frame, resize)
        frame = frame[:, :, [2, 1, 0]]
        frames.append(frame)

        if len(frames) == max_frames:
          break
    finally:
      video_capture.release()
    return np.array(frames) / 255.0

In [36]:
def save_frame_file(video_array, filename, start_frame, end_frame, word):
    s3_uri = "s3://asl-capstone/msasl/RGB/"
    filename = filename+".npy"
    if end_frame!=-1:
        video_array = video_array[int(start_frame):int(end_frame)].astype(np.float32)
    else:
        video_array = video_array.astype(np.float32)
        
    with fs.open(f"{s3_uri+filename}","wb") as f:
        np.save(f,video_array)
    try:
        with fs.open(f"s3://asl-capstone/msasl/master_file_Deanna_1.parquet","rb") as f:
            parquet_df = pd.read_parquet(f)
    except:
        parquet_df = pd.DataFrame()
        print("Parquet does not exist yet")
    temp_df = pd.DataFrame({'caption':[word], 'path':[s3_uri+filename]})
    parquet_df = pd.concat([parquet_df,temp_df], ignore_index=True)
    with fs.open(f"s3://asl-capstone/msasl/master_file_Deanna_1.parquet","wb") as f:
        parquet_df.to_parquet(f)

    

In [37]:
# deanna_collected_words = collected_words[10688:]
deanna_collected_words = collected_words[3993:5355]

iteration_1 = deanna_collected_words[:452]
iteration_2 = deanna_collected_words[452:908]
iteration_3 = deanna_collected_words[908:]


len(iteration_1)

452

In [40]:
%%time
filename_counter = {}
for video in tqdm(iteration_1[400:], desc="Processing videos"):
    try:
        word = video['word']
        url = video['url']
        start_frame = video['start']
        end_frame = video['end']
        if word in filename_counter:
            filename_counter[word]+=1
            filename = f"{word}_{filename_counter[word]}"
        else:
            filename_counter[word] = 1
            filename = f"{word}"
        #print(filename, url)
        downloaded_filename = filename+".mp4"
        download_vid(url, downloaded_filename);
        video_array = load_video(downloaded_filename);
        save_frame_file(video_array, filename, start_frame, end_frame, word);
        local_filename = "temp_folder/"+downloaded_filename
        os.remove(local_filename)
    except Exception as e:
        # print(f"Error with video download. Skipping it")
        print(e)

Processing videos:   0%|                                                                                               | 0/52 [00:00<?, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=sErq0TJMKEo
[youtube] sErq0TJMKEo: Downloading webpage
[youtube] sErq0TJMKEo: Downloading ios player API JSON
[youtube] sErq0TJMKEo: Downloading android player API JSON
[youtube] sErq0TJMKEo: Downloading m3u8 information
[info] sErq0TJMKEo: Downloading 1 format(s): 18
[download] Destination: temp_folder/everyday.mp4
[download] 100% of    9.31MiB in 00:00:00 at 27.52MiB/s    


Processing videos:   2%|█▋                                                                                     | 1/52 [00:15<13:33, 15.95s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=73icFhednQU
[youtube] 73icFhednQU: Downloading webpage
[youtube] 73icFhednQU: Downloading ios player API JSON
[youtube] 73icFhednQU: Downloading android player API JSON


ERROR: [youtube] 73icFhednQU: Private video. Sign in if you've been granted access to this video
Processing videos:   4%|███▎                                                                                   | 2/52 [00:16<05:50,  7.02s/it]

...

[youtube] Extracting URL: https://www.youtube.com/watch?v=Eq6SnaimpzQ
[youtube] Eq6SnaimpzQ: Downloading webpage
[youtube] Eq6SnaimpzQ: Downloading ios player API JSON
[youtube] Eq6SnaimpzQ: Downloading android player API JSON
[youtube] Eq6SnaimpzQ: Downloading m3u8 information
[info] Eq6SnaimpzQ: Downloading 1 format(s): 22
[download] Destination: temp_folder/excited_19.mp4
[download] 100% of   54.64MiB in 00:00:02 at 18.88MiB/s    


Processing videos:  98%|████████████████████████████████████████████████████████████████████████████████████▎ | 51/52 [14:02<00:21, 21.83s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=96HY0Pcl_e4
[youtube] 96HY0Pcl_e4: Downloading webpage
[youtube] 96HY0Pcl_e4: Downloading ios player API JSON
[youtube] 96HY0Pcl_e4: Downloading android player API JSON
[youtube] 96HY0Pcl_e4: Downloading m3u8 information
[youtube] 96HY0Pcl_e4: Downloading MPD manifest
[info] 96HY0Pcl_e4: Downloading 1 format(s): 22
[download] Destination: temp_folder/excited_20.mp4
[download] 100% of   54.80MiB in 00:00:01 at 29.07MiB/s    


Processing videos: 100%|██████████████████████████████████████████████████████████████████████████████████████| 52/52 [14:20<00:00, 16.55s/it]

CPU times: user 17min 47s, sys: 1min 56s, total: 19min 43s
Wall time: 14min 20s


In [ ]:
def get_instance_id():
    """Get current instance ID from metadata"""
    url = "http://169.254.169.254/latest/meta-data/instance-id"
    response = requests.get(url)
    return response.text

In [ ]:
def stop_instance(instance_id, region_name='us-west-2'):
    """Stop the EC2 instance"""
    ec2 = boto3.client('ec2', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key, region_name=region_name)
    ec2.stop_instances(InstanceIds=[instance_id])

In [ ]:
# Get the current instance ID
instance_id = get_instance_id()
print(instance_id)
# Stop the instance
stop_instance(instance_id)